In [ ]:
#from MuZeroParallel import MuZero as MuZeroP

from MuZero import MuZero

from GridWorldOption import GridWorldOption


from GridWorld import GridWorld

from mcts.MCTS import MCTS
import numpy as np
import matplotlib.pyplot as plt

from random import choice
from copy import deepcopy
import time

import pickle
import pandas as pd

from matplotlib.pyplot import figure
import itertools
from tqdm import tqdm

MAP_NAME = './maps/door.map'
with open(MAP_NAME) as f:
    the_map = f.read()
    
def get_g(full_options):
    env = GridWorld(the_map)

    G_model = {}
    for position in tqdm(env.possible_positions):
        for option in full_options:
            if option.is_valid(position):

                test_env = GridWorld(the_map)
                test_env.cur_x = position[0]
                test_env.cur_y = position[1]
                rewards = []
                while True:
                    action, should_break = option.get_action(test_env)

                    if action == -1:
                        break

                    next_position, r, _ = test_env.step(action)
                    rewards.append(r)

                    if should_break:
                        break

                G_model[(position, option)] = (next_position, rewards)
    return G_model

def get_v():
    V_model = {}

    env = GridWorld(the_map)
    the_opt  = GridWorldOption((env.goal_x, env.goal_y),   {'all'}, 10)

    for position in tqdm(env.possible_positions):
        env_test = GridWorld(the_map)

        the_opt  = GridWorldOption((env_test.goal_x, env_test.goal_y),   {'all'}, 10)


        env_test.cur_x = position[0]
        env_test.cur_y = position[1]

        state = position
        rewards = []
        done = False
        while True:

            action, _ = the_opt.get_action(env_test)

            if action == -1 or done:
                break

            #print(action)
            state, r, done = env_test.step(action)
            #print(state)
            rewards.append(r)

        G = 0
        counter = 0
        for r in rewards:
            G += (0.99 ** counter) * r
            counter += 1

        #print(len(rewards))
        #print(G)

        V_model[position] = G
    return V_model